# Curated Transactions with Merchant data (05)

In [21]:
import os
# get the accessKey and secretKey from Environment
accessKey = os.environ['AWS_ACCESS_KEY_ID']
secretKey = os.environ['AWS_SECRET_ACCESS_KEY']

from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
        .appName("Jupyter")
        .master("local[*]")

        .config("spark.jars.packages",
                "org.apache.iceberg:iceberg-spark-runtime-4.0_2.13:1.10.1,"
                "org.apache.iceberg:iceberg-aws-bundle:1.10.1")

        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.hadoop.fs.s3a.endpoint", "http://ibm-lh-presto-svc:9000")
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.access.key", "f33150f834d9a8b2435474f6")
        .config("spark.hadoop.fs.s3a.secret.key", "fdd7d613b2c72d07c3618ae6")
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

        # ==== Iceberg catalog (local) ===
        .config("spark.sql.catalog.hiverest", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.hiverest.type", "rest")
        .config("spark.sql.catalog.hiverest.uri", "http://hive-metastore:9084/iceberg")
        .config("spark.sql.catalog.hiverest.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .config("spark.sql.catalog.hiverest.warehouse", "s3a://admin-bucket/iceberg/warehouse")

        # ⭐ REQUIRED FOR MINIO WITH ICEBERG AWS SDK
        .config("spark.sql.catalog.hiverest.s3.endpoint", "http://minio-1:9000")
        .config("spark.sql.catalog.hiverest.s3.path-style-access", "true")
        .config("spark.sql.catalog.hiverest.s3.access-key-id", accessKey)
        .config("spark.sql.catalog.hiverest.s3.secret-access-key", secretKey)
    
        # ==== Iceberg catalog (watson) ===
        .config("spark.sql.catalog.watson", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.watson.type", "rest")
        .config("spark.sql.catalog.watson.uri", "https://ibm-lh-presto-svc:8180/mds/iceberg")
        .config("spark.sql.catalog.watson.rest.auth.type", "basic")
        .config("spark.sql.catalog.watson.rest.auth.basic.username", "ibmlhadmin")
        .config("spark.sql.catalog.watson.rest.auth.basic.password", "password")    
        .config("spark.sql.catalog.watson.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .config("spark.sql.catalog.watson.warehouse", "iceberg_data")

        # ⭐ REQUIRED FOR MINIO WITH ICEBERG AWS SDK
        .config("spark.sql.catalog.watson.s3.endpoint", "http://ibm-lh-presto-svc:9000")
        .config("spark.sql.catalog.watson.s3.path-style-access", "true")
        .config("spark.sql.catalog.watson.s3.access-key-id", "f33150f834d9a8b2435474f6")
        .config("spark.sql.catalog.watson.s3.secret-access-key", "fdd7d613b2c72d07c3618ae6")

        # use "hiverest" for local setup, "watson" for the watsonx developer setup
        .config("spark.sql.defaultCatalog", "hiverest")

        .config(
            "spark.driver.extraJavaOptions",
            "-Djavax.net.ssl.trustStore=/data-transfer/truststore.jks " +
            "-Djavax.net.ssl.trustStorePassword=changeit"
        )
        .config(
            "spark.executor.extraJavaOptions",
            "-Djavax.net.ssl.trustStore=/data-transfer/truststore.jks " +
            "-Djavax.net.ssl.trustStorePassword=changeit"
        )

        .config(
            "spark.sql.extensions",
            "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
        )

        .getOrCreate()
)

## Test SQL Statement using JupySQL

In [11]:
%load_ext sql
%config SqlMagic.displaylimit = 50
%sql spark

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [12]:
%%sql
describe payment_db.raw_transaction_t

Running query in 'SparkSession'

10 rows affected.

Field 1,Field 2,Field 3
transaction_id,string,None
card_number,string,None
currency,string,None
amount,double,None
merchant_id,string,None
channel,string,None
transaction_date,timestamp,None
,,
# Partitioning,,
Part 0,hours(transaction_date),


In [13]:
%%sql
SELECT t.transaction_id
    , t.card_number
    , t.currency
    , t.amount
    , t.channel
    , t.transaction_date
    , CONCAT(m.name, ' ', UPPER(m.city), ' ', m.country) AS booking_text
    , m.name
    , m.country
    , m.city
    , m.category_name
FROM payment_db.raw_transaction_t t
LEFT JOIN refdata_db.raw_merchant_t m
    ON (t.merchant_id = m.merchant_id);

Running query in 'SparkSession'

13878 rows affected.

Field 1,Field 2,Field 3,Field 4,Field 5,Field 6,Field 7,Field 8,Field 9,Field 10,Field 11
9b781a38-464e-eb3e-24f9-098df0741026,2221-2033-2398-1225,ILS,282.8699776498676,POS,2026-02-10 06:52:58.625000,Hills-Powlowski KRISTOFERMOUTH US,Hills-Powlowski,US,Kristofermouth,"Books, Computers & Home"
7c3ff156-c5c4-d2b5-b8b8-301a966c1608,6762-5415-0266-6040,CHF,117.96077431997422,POS,2026-02-10 06:52:58.831000,"Lindgren, Graham and Stamm WEST RUSSELTON US","Lindgren, Graham and Stamm",US,West Russelton,"Automotive, Games & Grocery"


## Using Spark SQL to execute SQL Join

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, upper, col

# Define the SQL query
query = """
SELECT t.transaction_id,
       t.card_number,
       t.currency,
       t.amount,
       t.channel,
       t.transaction_date,
       CONCAT(m.name, ' ', UPPER(m.city), ' ', m.country) AS booking_text,
       m.name,
       m.country,
       m.city,
       m.category_name
FROM payment_db.raw_transaction_t t
LEFT JOIN refdata_db.raw_merchant_t m
       ON t.merchant_id = m.merchant_id
"""

# Execute the query
result_df = spark.sql(query)

# Write to Iceberg table
result_df.write.format("iceberg") \
    .mode("overwrite") \
    .saveAsTable("payment_db.cur_transaction_with_merchant_sql_t")

In [15]:
%%sql
SELECT * FROM payment_db.cur_transaction_with_merchant_sql_t

Running query in 'SparkSession'

13878 rows affected.

Field 1,Field 2,Field 3,Field 4,Field 5,Field 6,Field 7,Field 8,Field 9,Field 10,Field 11
963866cf-69b9-cc37-a254-c651c71fd888,6304-1497-9301-9667,CAD,281.28384948900634,e-commerce,2026-02-10 06:00:00.581000,"Roberts, Spencer and Blick LAKE BELINDABERG US","Roberts, Spencer and Blick",US,Lake Belindaberg,Movies
73f16ccc-8dc6-929a-686f-a4f946f8f329,2720-8486-3192-4889,VED,157.0963392374642,POS,2026-02-10 06:00:00.711000,"Tillman, Gulgowski and Walsh NEW ZACKARYFORT US","Tillman, Gulgowski and Walsh",US,New Zackaryfort,Games


## Using PySpark to execute SELECT with JOIN

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, upper, col

# Read source tables
transactions_df = spark.table("payment_db.raw_transaction_t")
merchants_df = spark.table("refdata_db.raw_merchant_t")

# Join and compute new columns
result_df = transactions_df.alias("t") \
    .join(
        merchants_df.alias("m"),
        col("t.merchant_id") == col("m.merchant_id"),
        "left"
    ) \
    .select(
        col("t.transaction_id"),
        col("t.card_number"),
        col("t.currency"),
        col("t.amount"),
        col("t.channel"),
        col("t.transaction_date"),
        concat_ws(" ", col("m.name"), upper(col("m.city")), col("m.country")).alias("booking_text"),
        col("m.name"),
        col("m.country"),
        col("m.city"),
        col("m.category_name")
    )

# Write to Iceberg table
result_df.write.format("iceberg") \
    .mode("overwrite") \
    .saveAsTable("payment_db.cur_transaction_with_merchant_t")

In [17]:
%%sql
SELECT * FROM payment_db.cur_transaction_with_merchant_t

Running query in 'SparkSession'

13878 rows affected.

Field 1,Field 2,Field 3,Field 4,Field 5,Field 6,Field 7,Field 8,Field 9,Field 10,Field 11
11f89f11-48a9-72ec-e6c4-4c47e521e31e,5800-8407-6070-3005,TPE,133.8233449387126,ATM,2026-02-10 06:51:58.319000,Stiedemann and Sons RALEIGHFURT US,Stiedemann and Sons,US,Raleighfurt,Music
9dc67d5c-3060-8964-d009-88473443609a,5498-4289-3277-1889,KGS,74.68074723501077,e-commerce,2026-02-10 06:51:58.650000,"Becker, Wilkinson and Koch EAST CARMINABURY US","Becker, Wilkinson and Koch",US,East Carminabury,Beauty & Health


In [20]:
%%sql
show tables in payment_db

Running query in 'SparkSession'

3 rows affected.

Field 1,Field 2,Field 3
payment_db,raw_transaction_t,False
payment_db,cur_transaction_with_merchant_sql_t,False
payment_db,cur_transaction_with_merchant_t,False
